In [13]:
%pip install 'psycopg[binary]'

  Using cached psycopg-3.3.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached psycopg_binary-3.3.2-cp314-cp314-macosx_11_0_arm64.whl.metadata (2.7 kB)
Using cached psycopg-3.3.2-py3-none-any.whl (212 kB)
Using cached psycopg_binary-3.3.2-cp314-cp314-macosx_11_0_arm64.whl (4.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [psycopg]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import sys
from pathlib import Path
from decouple import config


In [2]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [25]:
from sqlmodel import Session, select, desc
from api.db.session import engine
from api.events.models import EventModel

In [29]:
with Session(engine) as session:
    query = select(EventModel).order_by(desc(EventModel.updated_at)).limit(10)
    compile_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compile_query)
    print("---")
    print(str(query))
    # results = session.exec(query).all()
    # print(results)

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at DESC
 LIMIT 10
---
SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at DESC
 LIMIT :param_1


In [ ]:
from timescaledb.hyperfunctions import time_bucket, time_bucket_gapfill
from pprint import pprint
from sqlalchemy import func
with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)
    query = select(bucket, EventModel.page, func.count()).group_by(bucket, EventModel.page).order_by(bucket.asc()).limit(10)
    compile_query = query.compile(compile_kwargs={"literal_binds": True})
    results = session.exec(query).all()
    pprint(results)

[(datetime.datetime(2026, 1, 12, 0, 0, tzinfo=datetime.timezone.utc), '/products', 1382),
 (datetime.datetime(2026, 1, 12, 0, 0, tzinfo=datetime.timezone.utc), '/contact', 1383),
 (datetime.datetime(2026, 1, 12, 0, 0, tzinfo=datetime.timezone.utc), '/careers', 1478),
 (datetime.datetime(2026, 1, 12, 0, 0, tzinfo=datetime.timezone.utc), '/blog', 1425),
 (datetime.datetime(2026, 1, 12, 0, 0, tzinfo=datetime.timezone.utc), '/about', 1425),
 (datetime.datetime(2026, 1, 12, 0, 0, tzinfo=datetime.timezone.utc), '/services', 1479),
 (datetime.datetime(2026, 1, 12, 0, 0, tzinfo=datetime.timezone.utc), '/home', 1428)]
